In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
df_asset = pd.read_parquet('Asset_summary_main.parquet')
df_news = pd.read_parquet('Quant_news_sentiment.parquet')

# Merge news sentiment absolute value from NLP model with asset price 

In [ ]:
df_news['Date'] = df_news['Timestamp'].dt.strftime('%Y%m%d')
df_news = df_news[(df_news['Timestamp'].dt.hour>=7) & (df_news['Timestamp'].dt.hour<= 11)] 
df_news[['title_sentiment_i', 'title_sentiment_i']] = df_news[['title_sentiment_i', 'title_sentiment_i']].abs()
df_news = df_news.groupby(pd.Grouper(key='Timestamp', freq='1min')).mean().reset_index()

df_asset = pd.merge(df_asset, df_news[['Timestamp', 'title_sentiment_i', 'description_sentiment_i']],on='Timestamp', how='left')
df_asset[['title_sentiment_i', 'description_sentiment_i']] = df_asset[['title_sentiment_i', 'description_sentiment_i']].fillna(0)


In [ ]:
def corr_analysis(df_filtered, rolling_period, start_hour = 7, end_hour = 11):
    df_filtered['JPY_return'] = np.log(df_asset.JPY_close) - np.log(df_asset.JPY_close.shift(1)) 
    df_filtered['EUR_return'] = np.log(df_asset.EUR_close) - np.log(df_asset.EUR_close.shift(1)) 
    df_filtered['TY1_return'] = np.log(df_asset.TY1_close) - np.log(df_asset.TY1_close.shift(1)) 
    df_filtered['ES1_return'] = np.log(df_asset.ES1_close) - np.log(df_asset.ES1_close.shift(1))

    df_filtered['JPY_EUR'] = df_filtered['JPY_return'].rolling(rolling_period).corr(df_filtered['EUR_return']) 
    df_filtered['JPY_TY1'] = df_filtered['JPY_return'].rolling(rolling_period).corr(df_filtered['TY1_return']) 
    df_filtered['JPY_SPX'] = df_filtered['JPY_return'].rolling(rolling_period).corr(df_filtered['ES1_return']) 
    df_filtered['EUR_TY1'] = df_filtered['EUR_return']. rolling(rolling_period). corr(df_filtered['TY1_return']) 
    df_filtered['EUR_SPX'] = df_filtered['EUR_return'].rolling(rolling_period).corr(df_filtered['ES1_return']) 
    df_filtered['TY1_SPX'] = df_filtered['TY1_return'].rolling(rolling_period).corr(df_filtered['ES1_return'])

    df_filtered = df_filtered[(df_filtered['Timestamp'].dt.hour>=start_hour) & (df_filtered['Timestamp'].dt.hour<= end_hour)] 
    return df_filtered

In [ ]:
df_corr = corr_analysis(df_asset, 30, start_hour = 7, end_hour = 11)
# df_corr

In [ ]:
fig = px.line(df_corr, x = 'Timestamp', y = ['JPY_EUR', 'JPY_TY1', 'JPY_SPX', 'EUR_TY1', 'EUR_SPX', 'TY1_SPX'])
fig

# Correlation reach extreme

In [ ]:
def pair_trade(Pair_selected, upper_corr_tag, lower_corr_tag):
    asset_1 = Pair_selected[:3]
    asset_2 = Pair_selected[-3:]

    for d in df_corr['Date'].unique():
        fig,axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
            
        data=df_corr[df_corr.Date == d].set_index('Timestamp')
        
        data['spread_return'] = (data[asset_1+'_return'] - data[asset_2+'_return'])*100 
        data['spread_return_accumulated'] = data['spread_return'].cumsum()

        # First correlation plot
        data[[Pair_selected]].plot(ax=axes[0])
        data[['spread_return_accumulated']].plot(secondary_y = True, ax=axes[0]) 
        axes[0].set_title(f'Correlation_plot_{d}')

        # Second asset price plot
        data[[asset_1+'_close']].plot(ax=axes[1])
        data[[asset_2+'_close']].plot(secondary_y = True, ax=axes[1]) 
        axes[1].set_title(f'Asset_price_plot_{d}')

        # Generate signals
        data.loc[data[Pair_selected] > upper_corr_tag, 'Signal'] = -1 # Short signal 
        data.loc[data[Pair_selected]< lower_corr_tag,'Signal'] = 1 # Long signal 
        
        signals = data['Signal']
        green_circles = signals[signals == 1] 
        red_triangles = signals[signals == -1]

        green_index = green_circles.index if len(green_circles.index)!=1 else green_circles.index[0] 
        red_index = red_triangles.index if len(red_triangles.index)!=1 else red_triangles.index[0] 
        
        # Add signals on plot
        axes[0].scatter(green_index, data.loc[green_index, Pair_selected], marker='o', color='green') 
        axes[0].scatter(red_index, data.loc[red_index, Pair_selected], marker='v', color='red')
        
        data[['title_sentiment_i', 'description_sentiment_i']].plot(ax=axes[2])
        axes[2].set_title(f'News_sentiment_{d}')


        fig.tight_layout() 
        plt.show()

    return 

In [ ]:
Pair_selected = 'JPY_EUR'
# Potential pair selection: 'JPY_EUR', 'JPY_TY1', 'JPY_SPX', 'EUR_TY1', 'EUR_SPX', 'TY1_SPX'
upper_corr_tag = -0.1
lower_corr_tag = -0.8

pair_trade(Pair_selected, upper_corr_tag, lower_corr_tag)